In [2]:
from pathlib import Path
import getpass
from time import time
import numpy as np
import os 
import csv
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import regularizers
from keras.models import Model
from keras.optimizers import SGD, Adamax
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint    

#!pip install keras==2.1.2 --upgrade
# Basic Guides:
# https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html



c:\users\jesper\appdata\local\conda\conda\envs\tensorflowv3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# configure setup
batch_size = 40
classes = 12
N_training = 12000
N_verification = 1000
dropout = 0.4
kernel_reg = 0.2
dense_size = 2048
train_layers = 229 #179 #229 #249
learning_rate = 1e-2
learning_rate_min = 1e-9
base_path = 'C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Logs/' 
custom_path = 'Run_v212_400_v1_TestRegFrom_v4'

# Create path
main_path = base_path + custom_path

# Create filename
filename = 'Par_T' + str(N_training) + '_V' + str(N_verification) + '_D' \
+ str(dense_size) + '_L' + str(train_layers) +'_B' + str(batch_size) + '_Drop' \
+ str(dropout).replace('.','_') + '_KReg' + str(kernel_reg).replace('.','_') \
+ '_Lr' + str(learning_rate).replace('.','_') + '_minLr' + str(learning_rate_min)

In [4]:
def load_data_plants(source):
    import shutil
    
    source1 = "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_400/train"
    dest11 =  "C:/Data/Workspace/GitHub/DeepLearningProject/LocalContent/Datasets/Resized_400/verf"
    folders = os.listdir(source)

    x_data = []
    
    for f in folders:
        files = os.listdir(source + '/'+ f)
        for i in files:
            x = image.load_img(source + '/'+ f + '/' + i)
            x = img_to_array(x)
            x_data.append(x)
    return x_data

# Class for modifying the tensorboard plotting
class TrainValTensorBoard(keras.callbacks.TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')        

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
      
        # Adding Learning rate to the logs
#         ittr = (self.model.optimizer.iterations)
#         decay = (self.model.optimizer.decay)
#         lr = (self.model.optimizer.lr)
#         beta_1 = (self.model.optimizer.beta_1)
        
#         lr *= (1. / (1. + decay * K.cast(ittr, K.dtype(decay))))
#         t = K.cast(ittr, K.floatx()) + 1
#         lr_t = lr / (1. - K.pow(beta_1, t))
        lr = (self.model.optimizer.lr)
        lr_t = lr
        
        
        logs['learning_rate'] = np.float64(K.eval(lr_t))
        logs['val_learning_rate'] = np.float64(K.eval(lr_t))
           
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [5]:
# Configure the loading of data
# Image Generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=360,
    #shear_range=0.3,
    zoom_range = 0.5,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    vertical_flip = True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        "../../LocalContent/Datasets/Resized_400/train",  # this is the target directory
        batch_size=batch_size,
        class_mode='categorical') 
# this is a similar generator, for validation data
validation_generator = val_datagen.flow_from_directory(
        "../../LocalContent/Datasets/Resized_400/verf",
        batch_size=batch_size,
        class_mode='categorical')

Found 3783 images belonging to 12 classes.
Found 967 images belonging to 12 classes.


In [6]:
%%capture
# Configure model
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a dropout layer
x = Dropout(rate = dropout)(x)
# let's add a fully-connected layer
x = Dense(dense_size, activation='relu',kernel_regularizer=regularizers.l2(kernel_reg))(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# Fit the model for the top layers
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done*after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# https://stackoverflow.com/questions/45943675/meaning-of-validation-steps-in-keras-sequential-fit-generator-parameter-list
model.fit_generator(
        train_generator,
        steps_per_epoch=N_training//batch_size,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=N_verification//batch_size,
        max_queue_size=1)

Epoch 1/1
300/300 [==============================] - 431s 1s/step - loss: 3.2129 - val_loss: 2.8389


In [7]:
# Load possible weights for the model
model.load_weights('C:/Data/Google Drive/DeepLearningDataStorage/Run_v212_299_v1_TestRegFrom_v4/Par_T7000_V1000_D2048_L229_B64_Drop0_4_KReg0_2_Lr0_01_minLr1e-09.E18-L0.37-A0.93.hdf5')

In [9]:
# Training Part
for layer in model.layers[:train_layers]: 
   layer.trainable = False
for layer in model.layers[train_layers:]:
   layer.trainable = True

model.compile(optimizer=SGD(lr=learning_rate, momentum=0.9, nesterov = True), loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adamax(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon()), loss='categorical_crossentropy', metrics=['accuracy'])


#ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.75,patience=3, min_lr=learning_rate_min,cooldown=1,verbose=1)
model_checker = ModelCheckpoint(main_path + '/' + filename + '.E{epoch:02d}-L{val_loss:.3f}-A{val_acc:.3f}.hdf5', 
                                monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)

#history = TrainValTensorBoard(log_dir=main_path + "/{}".format(time()), histogram_freq=0, write_graph=False, write_images=False)
history = TrainValTensorBoard(log_dir=main_path + '/' + filename, histogram_freq=0, write_graph=False, write_images=False)
#history = keras.callbacks.TensorBoard(log_dir="tmp/logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)


#tbCallBack = keras.callbacks.TensorBoard(log_dir="../../LocalContent/Logs/08_KerasInceptionV3_TestingDirectionLoading/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)


history = model.fit_generator(
        train_generator,
        steps_per_epoch=N_training//batch_size,
        epochs=500,
        validation_data=validation_generator,
        validation_steps=N_verification//batch_size,
        callbacks=[history,reduce_lr,model_checker],
        max_queue_size=5)#, workers=2, use_multiprocessing=True) #initial_epoch=76



Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/500
300/300 [==============================] - 593s 2s/step - loss: 1.5447 - acc: 0.5465 - val_loss: 0.6884 - val_acc: 0.8294
Epoch 2/500
300/300 [==============================] - 565s 2s/step - loss: 0.8035 - acc: 0.7954 - val_loss: 0.5255 - val_acc: 0.8873
Epoch 3/500
300/300 [==============================] - 565s 2s/step - loss: 0.6885 - acc: 0.8302 - val_loss: 0.4940 - val_acc: 0.8883
Epoch 4/500
300/300 [==============================] - 565s 2s/step - loss: 0.6295 - acc: 0.8488 - val_loss: 0.4825 - val_acc: 0.9007
Epoch 5/500
300/300 [==============================] - 563s 2s/step - loss: 0.5920 - acc: 0.8586 - val_loss: 0.3924 - val_acc: 0.9173
Epoch 6/500
300/300 [==============================] - 567s 2s/step - loss: 0.5741 - acc: 0.8638 - val_loss: 0.3804 - val_acc: 0.9338
Epoch 7/500
300/300 [==============================] - 565s 2s/step - loss: 0.5478 - acc: 0.8728 - val_loss: 0.3452 - val_a

KeyboardInterrupt: 

In [13]:
model.save_weights(main_path + '/Lastest.hdf5')